In [21]:
import numpy as np
import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from cooc_cross_validation import cooc_CV
from matplotlib.pyplot import plot
from glove_template import glove
from build_tweet_vectors import build_tweet_vector
from CV_glove import CV_glove
import pickle
import helpers
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
"""Paths"""
POS_TWEET_PATH = 'twitter-datasets/train_pos.txt'
NEG_TWEET_PATH = "twitter-datasets/train_neg.txt"
VOCAB_PATH = 'vocab.pkl'
GLOVE_EMB_PATH = "imported_embeddings/glove.twitter.27B.25d.npy"

BELOW decide which vocab to use

In [2]:
with open(VOCAB_PATH, 'rb') as file:
        vocab = pickle.load(file)

BElOW you decide which wordvector genereation to use

In [7]:
"""Bag of Word generation"""
X, Y = helpers.BOW_vector(vocab,POS_TWEET_PATH ,NEG_TWEET_PATH)

In [ ]:
"""GloVe emdeding"""
Xneg = build_tweet_vector(GLOVE_EMB_PATH,VOCAB_PATH,NEG_TWEET_PATH,"Embedded_tweets/embeddings_neg_imp_dim25.npy", TFID_weighting=False)
print("pos")
Xpos = build_tweet_vector(GLOVE_EMB_PATH,VOCAB_PATH,POS_TWEET_PATH,"Embedded_tweets/embeddings_pos_imp_dim25.npy", TFID_weighting = False)
Ypos = np.ones(Xpos.shape[0])
Yneg = np.ones(Xneg.shape[0])*-1
X = np.concatenate((Xneg,Xpos))
Y = np.concatenate((Yneg, Ypos))

In [24]:
"""TFID vector, generate needs to use Y from other generator right now not fixed
set old_vectorizer to false if you want to generate new one"""
old_vectorizer = False
fs = []
if old_vectorizer:
        with open('TFID.pkl', 'rb') as file:
                vectorizer = pickle.load(file)
else:
        vectorizer = TfidfVectorizer()
for fn in [POS_TWEET_PATH, NEG_TWEET_PATH]:
        with open(fn, errors= "ignore") as file:
                try: 
                        fs += file.readlines()
                except:
                        print(file.readlines())
print(len(fs))
if not old_vectorizer:
        vectorizer.fit(fs)
X = vectorizer.transform(fs)

200000


KeyboardInterrupt: 

Decide which supervised ML method to use:

In [15]:
"""Splits training data"""
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=1)

In [16]:
"""Multinomial naive bayes"""
GNB = MultinomialNB()
X_train = np.abs(X_train)
X_test = np.abs(X_test)
y_pred = GNB.fit(X_train, Y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"% (X_test.shape[0], (Y_test != y_pred).sum()))

Number of mislabeled points out of a total 100000 points : 23294
